In [4]:
pip install airflow

  Using cached airflow-0.6.tar.gz (1.2 kB)
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\pc\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\pc\\AppData\\Local\\Temp\\pip-install-9mqj8861\\airflow_d647da59d514411292a0e443bc0e38b1\\setup.py'"'"'; __file__='"'"'C:\\Users\\pc\\AppData\\Local\\Temp\\pip-install-9mqj8861\\airflow_d647da59d514411292a0e443bc0e38b1\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\pc\AppData\Local\Temp\pip-pip-egg-info-gaaght8s'
         cwd: C:\Users\pc\AppData\Local\Temp\pip-install-9mqj8861\airflow_d647da59d514411292a0e443bc0e38b1\
    Complete output (13 lines):
    running egg_info
    creating C:\Users\pc\AppData\Local\Temp\pip-pip-egg-info-gaaght8s\airflow.egg-info
    

  Using cached apache_airflow-2.7.1-py3-none-any.whl (12.9 MB)
  Using cached cattrs-23.1.2-py3-none-any.whl (50 kB)
  Using cached colorlog-4.8.0-py2.py3-none-any.whl (10 kB)
  Using cached opentelemetry_api-1.20.0-py3-none-any.whl (57 kB)
  Using cached rfc3339_validator-0.1.4-py2.py3-none-any.whl (3.5 kB)
  Using cached croniter-1.4.1-py2.py3-none-any.whl (19 kB)
  Using cached connexion-2.14.2-py2.py3-none-any.whl (95 kB)
  Using cached Flask_Login-0.6.2-py3-none-any.whl (17 kB)
  Using cached mdit_py_plugins-0.4.0-py3-none-any.whl (54 kB)
  Using cached SQLAlchemy_JSONField-1.0.1.post0-py3-none-any.whl (10 kB)
  Using cached cron_descriptor-1.4.0-py3-none-any.whl
  Using cached flask_session-0.5.0-py3-none-any.whl (7.2 kB)
  Using cached apache_airflow_providers_imap-3.3.2-py3-none-any.whl (17 kB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Using cached setproctitle-1.3.2-cp39-cp39-win_amd64.whl (11 kB)
  Using cached google_re2-1.1-2-cp39-cp39-win_amd64.whl (476 kB)
  U

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.7.1 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.4 which is incompatible.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.


In [6]:
pip install apache-airflow-providers-microsoft-mssql

Note: you may need to restart the kernel to use updated packages.


In [7]:
import time
from datetime import datetime
from airflow.decorators import dag, task
from airflow.providers.microsoft.mssql.hooks.mssql import MsSqlHook
import pandas as pd
from google.oauth2 import service_account

[2023-09-25T21:00:01.305+0100] {utils.py:159} INFO - NumExpr defaulting to 8 threads.


In [20]:
@dag(schedule_interval="0 10 * * *", start_date=datetime(2023, 9, 25), catchup=False, tags=['load_gcp'])
def extract_and_load():
    @task()
    def sql_extract():
        try:
            hook = MsSqlHook(mssql_conn_id="sqlserver")
            sql = """  SELECT t.name AS table_name
                        FROM sys.tables t
                        WHERE t.name IN ('sales'); """
            df = hook.get_pandas_df(sql)
            print(df)
            tbl_dict = df.to_dict('dict')
            return tbl_dict
        except Exception as e:
            print("Data extract error: " + str(e))
    
    @task()
    def gcp_load(tbl_dict: dict):
        #
        try:
            credentials = service_account.Credentials.from_service_account_file( 'vocal-raceway-400113-7e5d84b42be1.json')
            project_id = "your_project_id" #you can get it from your Project properties on GCP (bigQuery)
            dateset_ref = "your_data_ref" #you can get it from your dataset properties on GCP (bigQuery)
            
            for value in tbl_dict.values():
                val = value.values()
                for v in val:
                    print(v)
                    rows_imported = 0
                    sql = f'select * FROM {v}'
                    hook = MsSqlHook(mssql_conn_id="sqlserver")
                    df = hook.get_pandas_df(sql)
                    print(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {v} ')
                    df.to_gbq( destination_table=f'{dateset_ref}.src_{v}',  project_id=project_id, credentials=credentials, if_exists="replace" )
                    rows_imported += len(df)
        except Exception as e:
            print("Data load error: " + str(e))
    tbl_dict = sql_extract()
    tbl_summary = gcp_load(tbl_dict)

gcp_extract_and_load = extract_and_load()
print(gcp_extract_and_load)

<DAG: extract_and_load>
